# Introduction
This notebook contains the steps to complete day 1 of Dashboarding with Notebooks tutorial by Rachael Tatman at Kaggle. The instructions are detailed in day-1-instructions.ipynb.

# Dataset selection



In [ ]:
# Download and unzip data.

# Data preparation
There are two steps to prepare the data for visualisation: reading the data and cleaning it.

## Reading data


In [1]:
from pathlib import Path  # Easy-to-use, cross-platform path-to-file.

import numpy as np
import pandas as pd

## Data cleaning



# Visualisation

In [2]:
import matplotlib.pyplot as plt  # Quick plotting.
import seaborn  # Make plotting better.
%matplotlib inline